In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import gc

# Function to convert ROC date to Gregorian date
def roc_to_gregorian(date_int):
    try:
        date_str = str(int(date_int))
        if len(date_str) == 7:  # YYYMMDD
            year = int(date_str[:3]) + 1911
            date_str = str(year) + date_str[3:]
        elif len(date_str) == 6:  # YYMMDD
            year = int(date_str[:2]) + 1911
            date_str = str(year) + date_str[2:]
        else:
            return pd.NaT
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    except:
        return pd.NaT

# Function to parse '交易筆棟數'
def parse_transaction_numbers(s):
    import re
    pattern = r'土地(\d+)建物(\d+)車位(\d+)'
    match = re.search(pattern, s)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    else:
        return np.nan, np.nan, np.nan

# Read data
train_data = pd.read_excel("/kaggle/input/machine-learning-2024-iaii-regression/train.xlsx", engine='openpyxl')
valid_data = pd.read_excel("/kaggle/input/machine-learning-2024-iaii-regression/valid.xlsx", engine='openpyxl')
test_data = pd.read_excel("/kaggle/input/machine-learning-2024-iaii-regression/test-reindex-test.xlsx", engine='openpyxl')

# Combine train and valid data for cross-validation
train_data = pd.concat([train_data, valid_data], ignore_index=True)

# Drop unnecessary columns
cols_to_drop = ['建築完成年月', '解約情形', '土地位置建物門牌', '備註', '棟及號', '有無管理組織', '移轉層次']
train_data = train_data.drop(columns=cols_to_drop + ['編號'])
test_data = test_data.drop(columns=cols_to_drop)  # Keep '編號' in test_data

# Save '編號' for submission
test_ids = test_data['編號'].copy()
test_data = test_data.drop(columns=['編號'])

# Concatenate all data for consistent preprocessing
full_data = pd.concat([train_data, test_data], keys=['train', 'test'], ignore_index=False)

# Convert target variable to numeric
full_data['總價元'] = pd.to_numeric(full_data['總價元'], errors='coerce')

# Convert '交易年月日' to datetime
full_data['交易年月日'] = full_data['交易年月日'].apply(roc_to_gregorian)

# Extract date features
full_data['交易年'] = full_data['交易年月日'].dt.year
full_data['交易月'] = full_data['交易年月日'].dt.month
full_data['交易日'] = full_data['交易年月日'].dt.day

# Handle missing values in date features
full_data['交易年'] = full_data['交易年'].fillna(full_data['交易年'].mode()[0])
full_data['交易月'] = full_data['交易月'].fillna(full_data['交易月'].mode()[0])
full_data['交易日'] = full_data['交易日'].fillna(full_data['交易日'].mode()[0])

# Handle missing values for numerical columns
numerical_cols = ['土地移轉總面積平方公尺', '總樓層數', '建物移轉總面積平方公尺',
                  '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛',
                  '單價元平方公尺', '車位移轉總面積平方公尺', '車位總價元']

for col in numerical_cols:
    full_data[col] = pd.to_numeric(full_data[col], errors='coerce')
    full_data[col] = full_data[col].fillna(full_data[col].median())

# Log transform skewed numerical features
skewed_cols = ['土地移轉總面積平方公尺', '建物移轉總面積平方公尺', '單價元平方公尺', '車位移轉總面積平方公尺', '車位總價元']
for col in skewed_cols:
    full_data[col] = full_data[col].apply(lambda x: np.log1p(x) if x > 0 else 0)

# Parse '交易筆棟數'
full_data[['土地數', '建物數', '車位數']] = full_data['交易筆棟數'].apply(
    lambda x: pd.Series(parse_transaction_numbers(x))
)
full_data = full_data.drop(columns=['交易筆棟數'])

# Fill missing values in parsed columns
full_data['土地數'] = full_data['土地數'].fillna(full_data['土地數'].median())
full_data['建物數'] = full_data['建物數'].fillna(full_data['建物數'].median())
full_data['車位數'] = full_data['車位數'].fillna(full_data['車位數'].median())

# Create new features
full_data['總房間數'] = full_data['建物現況格局-房'] + full_data['建物現況格局-廳'] + full_data['建物現況格局-衛']
full_data['房廳比例'] = full_data['建物現況格局-房'] / (full_data['建物現況格局-廳'] + 1)
full_data['房衛比例'] = full_data['建物現況格局-房'] / (full_data['建物現況格局-衛'] + 1)

# Handle missing values for categorical columns
categorical_cols = ['鄉鎮市區', '交易標的', '都市土地使用分區', '非都市土地使用分區',
                    '非都市土地使用編定', '建物型態', '主要用途', '主要建材',
                    '建物現況格局-隔間', '車位類別', '建案名稱']
for col in categorical_cols:
    full_data[col] = full_data[col].fillna('Unknown')

# **Convert categorical columns to numeric (Label Encoding)**
label_enc = LabelEncoder()
for col in categorical_cols:
    full_data[col] = label_enc.fit_transform(full_data[col].astype(str))

# Separate the data back
train_df = full_data.loc['train']
test_df = full_data.loc['test']

# Drop rows with missing target
train_df = train_df[train_df['總價元'].notnull()]

# Features and target
X = train_df.drop(columns=['總價元', '交易年月日'])
y = train_df['總價元']

X_test = test_df.drop(columns=['總價元', '交易年月日'])

# Cross-validation and model training
folds = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(X.shape[0])
test_preds = np.zeros(X_test.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print(f'Fold {fold_ + 1}')
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'learning_rate': 0.01,
        'verbose': -1,
        'n_jobs': -1,
        'seed': 42,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'bagging_freq': 1,
    }

    clf = lgb.train(
        params,
        trn_data,
        num_boost_round=10000,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=200), lgb.log_evaluation(500)]
    )

    oof_preds[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)
    test_preds += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

    del trn_data, val_data, clf
    gc.collect()

# Evaluate performance without log transformation (no need for expm1)
mae = mean_absolute_error(y, oof_preds)
print(f'Overall MAE: {mae}')

# Since we didn't log-transform the target, there's no need to apply expm1
# Simply use the test_preds directly

# Prepare submission file
submission = pd.DataFrame({'編號': test_ids.astype(int), '總價元': test_preds.round().astype(int)})
submission.to_csv('submission.csv', index=False, encoding='utf-8-sig')



Fold 1
Training until validation scores don't improve for 200 rounds
[500]	valid_0's l1: 663241
[1000]	valid_0's l1: 456020
[1500]	valid_0's l1: 408554
[2000]	valid_0's l1: 384729
[2500]	valid_0's l1: 372956
[3000]	valid_0's l1: 362556
[3500]	valid_0's l1: 355213
[4000]	valid_0's l1: 348189
[4500]	valid_0's l1: 342629
[5000]	valid_0's l1: 338947
[5500]	valid_0's l1: 335139
[6000]	valid_0's l1: 332441
[6500]	valid_0's l1: 330225
[7000]	valid_0's l1: 327024
[7500]	valid_0's l1: 323695
[8000]	valid_0's l1: 321006
[8500]	valid_0's l1: 319281
[9000]	valid_0's l1: 317146
[9500]	valid_0's l1: 315215
[10000]	valid_0's l1: 313011
Did not meet early stopping. Best iteration is:
[9970]	valid_0's l1: 312969
Fold 2
Training until validation scores don't improve for 200 rounds
[500]	valid_0's l1: 675229
[1000]	valid_0's l1: 467729
[1500]	valid_0's l1: 418363
[2000]	valid_0's l1: 396885
[2500]	valid_0's l1: 381290
[3000]	valid_0's l1: 369404
[3500]	valid_0's l1: 360801
[4000]	valid_0's l1: 354441
[45